### Quset 230828
- BreastCancerWisconsinDataSet.csv 연속형 데이터 사용
- 목표변수(diagnosis), 설명변수(*_mean아닌 것 선택 3개)
- diagnosis 전처리(appy() M-0, B-1), 
- Logistic regression 학습, 평가, 서비스(pickle)

In [1]:
import pandas as pd

In [2]:
df_BCWD = pd.read_csv('../../../datasets/BreastCancerWisconsinDataset.csv')
df_BCWD[:2]

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN


####  목표변수 설정
- 목표변수 : diagnosis
- 설명변수 :  area_se , radius_se , texture_se

In [3]:
df_BCWD.columns 

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [4]:
df_BCWD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [26]:
df_BCWD_extract = df_BCWD[['diagnosis', 'radius_se', 'texture_se','area_se']]

In [27]:
def change_diagnosis_value(value):
    if value == 'M':
        return 0
    elif value == 'B':
        return 1
    else:
        return value

# 'diagnosis' 컬럼에 함수 적용하여 값 변경
df_BCWD_extract['diagnosis'] = df_BCWD_extract['diagnosis'].apply(change_diagnosis_value)
df_BCWD_extract

C:\Users\04-04\AppData\Local\Temp\ipykernel_14048\844848198.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BCWD_extract['diagnosis'] = df_BCWD_extract['diagnosis'].apply(change_diagnosis_value)


,diagnosis,radius_se,texture_se,area_se
0,0,1.0950,0.9053,153.40
1,0,0.5435,0.7339,74.08
2,0,0.7456,0.7869,94.03
3,0,0.4956,1.1560,27.23
4,0,0.7572,0.7813,94.44
...,...,...,...,...
564,0,1.1760,1.2560,158.70
565,0,0.7655,2.4630,99.04
566,0,0.4564,1.0750,48.55
567,0,0.7260,1.5950,86.22


In [28]:
df_BCWD_extract.isnull().sum()

diagnosis     0
radius_se     0
texture_se    0
area_se       0
dtype: int64

In [29]:
df_BCWD_extract_preprocess = df_BCWD_extract.dropna()
df_BCWD_extract_preprocess

,diagnosis,radius_se,texture_se,area_se
0,0,1.0950,0.9053,153.40
1,0,0.5435,0.7339,74.08
2,0,0.7456,0.7869,94.03
3,0,0.4956,1.1560,27.23
4,0,0.7572,0.7813,94.44
...,...,...,...,...
564,0,1.1760,1.2560,158.70
565,0,0.7655,2.4630,99.04
566,0,0.4564,1.0750,48.55
567,0,0.7260,1.5950,86.22


#### 정형화 단계

In [30]:
target = df_BCWD_extract_preprocess['diagnosis']
labels = df_BCWD_extract_preprocess[['radius_se','texture_se','area_se']]
target.shape, labels.shape

((569,), (569, 3))

#### 모델학습

In [31]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(labels,target)

LogisticRegression()

In [33]:
model.coef_, model.intercept_

(array([[ 2.23214843,  1.07334987, -0.1622489 ]]), array([3.69375892]))

#### 평가

In [34]:
df_BCWD_extract_preprocess[10:15] # index가 features_train 같다.

,diagnosis,radius_se,texture_se,area_se
10,0,0.3795,1.1870,40.51
11,0,0.5058,0.9849,54.16
12,0,0.9555,3.5680,116.20
13,0,0.4033,1.0780,36.58
14,0,0.2121,1.1690,19.21


In [35]:
model.predict(labels[10:15])

array([0, 0, 0, 0, 1], dtype=int64)

In [36]:
model.predict_proba(labels[10:15])

array([[6.80870960e-01, 3.19129040e-01],
       [9.48214884e-01, 5.17851163e-02],
       [9.99898671e-01, 1.01329389e-04],
       [5.45872914e-01, 4.54127086e-01],
       [9.06979731e-02, 9.09302027e-01]])

In [37]:
target_predict = model.predict(labels)
# target_predict

In [38]:
from sklearn.metrics import accuracy_score    # 정확도

In [39]:
accuracy_score(target, target_predict)

0.8910369068541301

#### 서비스 배포

In [40]:
import pickle

In [42]:
with open('../../../datasets/BreastCancerWisconsinDataSet.pkl','wb') as regression_file : 
    pickle.dump(obj=model, file=regression_file)
    pass